Questions to ask:
 - What is the age distribution and is it normal?
 - Is the variables correlated?
 - Can the images of the smoker and non-smoker be easily distinguished?

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nltk.tokenize import RegexpTokenizer
from wordcloud import WordCloud
from colorama import Fore, Back, Style #color style


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Importing Machine Learning and Deep Learning Modules
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics


import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

from tqdm import tqdm
from PIL import Image
import pydicom

In [ ]:
!pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks

In [ ]:
# Statistical Packages
from scipy.stats import shapiro
from scipy import stats 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
import pickle
import gc
import pydicom


In [ ]:
!conda install -y gdcm -c conda-forge

In [ ]:
# Reading the Dataset
train_fibro = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/train.csv')
test_fibro = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')
sample_submission_fibro = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/sample_submission.csv')

In [ ]:
# First rows of train dataset
train_fibro.head()

In [ ]:
# First rows of test dataset
test_fibro.head()

In [ ]:
# First rows of sample submission dataset
sample_submission_fibro.head()

In [ ]:
# Looking at the smoking status to sex
train_fibro.groupby(['SmokingStatus']).count()['Sex'].to_frame()

In [ ]:
# Age status to sex
train_fibro.groupby(['Age']).count()['Sex'].to_frame()

In [ ]:
# Age Distribution
plt.hist(train_fibro['Age'], color='red')

In [ ]:
# Creating the Deviation Column
train_fibro["Deviation"] = train_fibro['FVC']/train_fibro['Percent']
test_fibro['Deviation'] = test_fibro['FVC']/test_fibro['Percent']

In [ ]:
# Looking at the train_fibro dataset again
train_fibro.head()

In [ ]:
# Looking at the test_fibro dataset again
test_fibro.head()

In [ ]:
# Histogram of train_fibro deviation
plt.hist(train_fibro['Deviation'], color='blue')

In [ ]:
# Looking at Missing Values
train_fibro.isnull().sum() #Training dataset
sns.heatmap(train_fibro.isnull(), cbar='Magma')

In [ ]:
test_fibro.isnull().sum() #testing dataset

sns.heatmap(test_fibro.isnull(), cbar=False)

# This dataset has no missing values, let's march on to the analysis.

In [ ]:
# Train Correlation
train_fibro.corr()

In [ ]:
# Train Correlation Matrix
corr_matrix = train_fibro.corr() 
f, ax = plt.subplots(figsize =(15, 13)) 
sns.heatmap(corr_matrix, ax = ax, cmap = 'viridis', linewidths = 0.5) 

In [ ]:
# Test Correlation
test_fibro.corr()

In [ ]:
# Test Correlation Heatmap
corr_matrix = test_fibro.corr() 
f, ax = plt.subplots(figsize =(15, 13)) 
sns.heatmap(corr_matrix, ax = ax, cmap = 'magma', linewidths = 0.5) 

The test dataset has a higher correlation between the percent vs. weeks. The same result is happening with the train dataset as well.

In [ ]:
# Looking for unique values
train_patient_id = set(train_fibro['Patient'].unique())
test_patient_id = set(test_fibro['Patient'].unique())

train_patient_id.intersection(test_patient_id)

In [ ]:
# Looking at the counts for the training dataset
print(train_fibro['Patient'].count(),train_fibro['Patient'].value_counts().shape[0])

In [ ]:
# Testing dataset counts
print(test_fibro['Patient'].count(),test_fibro['Patient'].value_counts().shape[0])

In [ ]:
train_patient_ids = set(train_fibro['Patient'].unique())
test_patient_ids = set(test_fibro['Patient'].unique())

train_patient_ids.intersection(test_patient_ids)

In [ ]:
# Getting the column names on the training dataset
train_fibro_col = train_fibro.keys()
train_fibro_col = list(train_fibro_col)
print(train_fibro_col)

In [ ]:
# Testing dataset for the column names
test_fibro_col = test_fibro.keys()
test_fibro_col = list(test_fibro_col)
print(test_fibro_col)

In [ ]:
# Getting the Unique Patient counts
train_fibro['Patient'].value_counts().max()


In [ ]:
# Testing version of it
test_fibro['Patient'].value_counts().max()


In [ ]:
# Getting the 95% of confidence interval of the values for the patient
print(np.quantile(train_fibro['Patient'].value_counts(), 0.95))
print(np.quantile(test_fibro['Patient'].value_counts(), 0.95))

In [ ]:
# Age differences
np.quantile(train_fibro['Age'].value_counts(), 0.75) - np.quantile(test_fibro['Age'].value_counts(), 0.25)


In [ ]:
# Getting the value counts for the differences
np.quantile(train_fibro['Patient'].value_counts(), 0.75) - np.quantile(test_fibro['Patient'].value_counts(), 0.25)


In [ ]:
# Looking at the PVC Variable at 5% significance
stat, p_value = shapiro(train_fibro['FVC'])
print('Statistics=%.3f, p=%.3f' % (stat, p_value))
# interpret
alpha = 0.05
if p_value > alpha:
    print('Sample resembles to Gaussian (fail to reject H0)')
else:
    print('reject H0')


In [ ]:
# Fitting Lambda Values

_, lambda_fit = stats.boxcox(train_fibro['FVC']) 
print("Lambda Values: ", lambda_fit)


In [ ]:
# Box-Cox Functions
def bctransform(x, lamb):
    part1 = x**lamb
    part2 = part1-1
    result = part2/lamb
    return result

def reversebct(x, lamb):
    x = np.where(x<0,0,x)
    part1 = x*lamb + 1
    result = part1**(1/lamb)
    return result

In [ ]:
# Reverse Box Cox Transform
train_fitted = bctransform(train_fibro['FVC'], lamb=lambda_fit)
plt.hist(reversebct(train_fitted, lamb=lambda_fit))

In [ ]:
# Shapiro Test on Box Cox Reverse
stat, p = shapiro(train_fitted)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
    print('Fail to reject H0')
else:
    print('Reject H0')


In [ ]:
# Plotting distributions of fitted data in the training dataset
plt.hist(reversebct(train_fitted, lamb=lambda_fit))


In [ ]:
# Boxplot
trace0 = go.Box(y=train_fibro["Age"],name="Age")
trace1 = go.Box(y=train_fibro["Percent"],name="Percent")


tracer = [trace0,  trace1]
iplot(tracer)

In [ ]:
# Looking at the weeks distribution on the training dataset
train_fibro['Weeks'].value_counts().head()

In [ ]:
# Looking at the weeks distribution on the test dataset
test_fibro['Weeks'].value_counts().head()

In [ ]:
# Looking at how long the patient stays for lung treatment
fibro_weeks = train_fibro.groupby("Patient").agg({"Weeks":"nunique","Age":"nunique"}).reset_index()
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12,8))
sns.countplot(fibro_weeks.Weeks,ax = ax1);
sns.countplot(fibro_weeks.Age,ax =ax2);


Week 9 has the highest counts for the patient recordings in the lung treatments. Each of them has 1 year age of difference.

# Weeks Distribution

In [ ]:
train_fibro['Weeks'].hist(color="orange")
plt.xlabel('Weeks') 
plt.ylabel('Count') 
plt.title('Training Set Weeks Distribution', 
          fontweight ="bold") 
  
plt.show() 

In [ ]:
# Age Distribution
train_fibro_patients = train_fibro[["Patient","Sex","SmokingStatus","Age"]].drop_duplicates()
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(20, 5),gridspec_kw={'width_ratios': [1,1,2]})
sns.countplot(train_fibro_patients.Sex,ax = ax1);
sns.countplot(train_fibro_patients.SmokingStatus,ax =ax2);
sns.countplot(train_fibro_patients.Age,ax =ax3);


In [ ]:
# Scatter Plot of Weeks vs. Age
figure = px.scatter(train_fibro, x="Weeks", y="Age", color='Sex')
figure.show()


In [ ]:
# Bootstrap Preparation
train_fibro_boot_dev = np.array(train_fibro['Deviation'])

In [ ]:
# Creating Bootstrap Model
train_fibro_bootstrap = []
for i in range(10000):
    np.random.seed(i)
    train_fibro_bootstrap.append((resample(train_fibro_boot_dev)))
print(len(train_fibro_bootstrap))

In [ ]:
# Calculating Bootstrap Means
bootstrap_means = np.mean(train_fibro_bootstrap, axis=1)
bootstrap_means

In [ ]:
# Histogram of Bootstrap Means
lower_bound = np.percentile(bootstrap_means, 2.5)
upper_bound = np.percentile(bootstrap_means, 97.5)

fig = plt.figure(figsize=(10,3))
ax = plt.hist(bootstrap_means, bins=30)

plt.xlabel('Age')
plt.ylabel('Frequency')
plt.axvline(lower_bound, color='r')
plt.axvline(upper_bound, color='r')
plt.show()

print('Lower bound: {}'.format(lower_bound))
print('Upper bound: {}'.format(upper_bound))

In [ ]:
# Installing Fastai
!pip install fastai2 -q

In [ ]:
# Looking at the X-Ray Images
train_image_data = "../input/osic-pulmonary-fibrosis-progression/train"


In [ ]:
# Looking up DCM Files
print(Fore.GREEN + 'Train .dcm number of images:',Style.RESET_ALL, len(list(os.listdir('../input/osic-pulmonary-fibrosis-progression/train'))), '\n' +
      Fore.BLUE + 'Test .dcm number of images:',Style.RESET_ALL, len(list(os.listdir('../input/osic-pulmonary-fibrosis-progression/test'))), '\n' +
      '--------------------------------', '\n' +
      'There is the same number of images as in train/ test .csv datasets')

In [ ]:
def pix_array(dataset, figsize=(10,8)):
    plt.figure(figsize=figsize)
    plt.grid(False)
    plt.imshow(dataset.pix_array, cmap='gray') # cmap=plt.cm.bone (color))
    plt.show()

In [ ]:
# Smoker
dir_image = "../input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430"
print("total images for patient ID00007637202177411956430: ", len(os.listdir(dir_image)))

# view first (columns*rows) images in order
fig=plt.figure(figsize=(14, 14))
columns = 5
rows = 4
imglist = os.listdir(dir_image)
for i in range(1, columns*rows +1):
    filename = dir_image + "/" + str(i) + ".dcm"
    ds = pydicom.dcmread(filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(ds.pixel_array, cmap='gray')
plt.show()

In [ ]:
# Patient ID00426637202313170790466 (Non-Smoker)

dir_image = "../input/osic-pulmonary-fibrosis-progression/test/ID00426637202313170790466"
print("total images for patient ID00426637202313170790466: ", len(os.listdir(dir_image)))

# view first (columns*rows) images in order
fig=plt.figure(figsize=(14, 14))
columns = 5
rows = 4
imglist = os.listdir(dir_image)
for i in range(1, columns*rows +1):
    filename = dir_image + "/" + str(i) + ".dcm"
    ds = pydicom.dcmread(filename)
    fig.add_subplot(rows, columns, i)
    plt.imshow(ds.pixel_array, cmap='gray')
plt.show()

# References:
- https://www.kaggle.com/piantic/osic-pulmonary-fibrosis-progression-basic-eda
- https://www.kaggle.com/yeayates21/osic-image-data-prep-and-baseline-regression-model
- https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/
